# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import requests
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import os
import re
import string
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, KFold,cross_val_predict
from sklearn.preprocessing import RobustScaler
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
y = df.iloc[:,4:]
columns_names = y.columns

### 2. Write a tokenization function to process your text data

In [3]:

def tokenize(text):
    '''
    This process clean the message by tokenize and lemmatize
    
    INPUT: df with the message

    OUTPUT: df modified 
    '''
    # this part handle some part of the special characters

    text= re.sub(r"[^a-zA-Z0-9]"," ",text)

    # this part tokenize and remove stopwords and other class of special characters
    text_tokenize = word_tokenize(text)
    text_tokenize = [y for y in text_tokenize if y not in stopwords.words('english')+list(string.punctuation)]

    #define to lemmatize
    lemmatizer=WordNetLemmatizer()

    #This lemmatize the input
    clean_tokens = []
    for tok in text_tokenize:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    #get the clean information
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)

In [6]:
pipeline_fit = pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline_fit.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [8]:
accuracy_mean = []
f1_mean = []
precision_mean = []
recall_mean = []

for i in range(0,len(columns_names.shape),1):
    accuracy_mean.append(accuracy_score(y_test.iloc[:,i], y_pred.iloc[:,i]))
    f1_mean.append(f1_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))
    precision_mean.append(precision_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))
    recall_mean.append(recall_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))

accuracy_mean = pd.DataFrame(accuracy_mean)
f1_mean = pd.DataFrame(f1_mean)
precision_mean = pd.DataFrame(precision_mean)
recall_mean = pd.DataFrame(recall_mean)

print('The mean accuracy score of the model is',accuracy_mean.mean())
print('The mean precision score of the model is',precision_mean.mean())
print('The mean recall of the model is',recall_mean.mean())
print('The mean F1 score of the model is',f1_mean.mean())

The mean accuracy score of the model is 0    0.809497
dtype: float64
The mean precision score of the model is 0    0.684744
dtype: float64
The mean recall of the model is 0    0.558915
dtype: float64
The mean F1 score of the model is 0    0.599806
dtype: float64


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
#establish some parameters for improve del model
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

parameters  = { "clf__estimator__n_estimators": [50,100,150], 
                "clf__estimator__max_depth" : [1,2,3,4,5,6],
                "clf__estimator__max_samples": [0.05, 0.1],
                'clf__estimator__random_state':[42]}

cv = GridSearchCV(pipeline,param_grid=parameters,cv=3)

In [ ]:
model = cv.fit(X_train,y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = model.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [ ]:
accuracy_mean = []
f1_mean = []
precision_mean = []
recall_mean = []

for i in range(0,len(columns_names.shape),1):
    accuracy_mean.append(accuracy_score(y_test.iloc[:,i], y_pred.iloc[:,i]))
    f1_mean.append(f1_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))
    precision_mean.append(precision_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))
    recall_mean.append(recall_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))

accuracy_mean = pd.DataFrame(accuracy_mean)
f1_mean = pd.DataFrame(f1_mean)
precision_mean = pd.DataFrame(precision_mean)
recall_mean = pd.DataFrame(recall_mean)

print('The mean accuracy score of the model is',accuracy_mean.mean())
print('The mean precision score of the model is',precision_mean.mean())
print('The mean recall of the model is',recall_mean.mean())
print('The mean F1 score of the model is',f1_mean.mean())

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
#establish some parameters for improve del model
pipeline2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
    ])),

    ('ada', MultiOutputClassifier(AdaBoostClassifier()))
])

#parameters2  = {'ada__estimator__n_estimators' : [100,150],
#                'ada__estimator__random_state':[42]}

#cv2 = GridSearchCV(pipeline2, param_grid=parameters2)


In [19]:
model2 = pipeline2.fit(X_train,y_train)

In [20]:
y_pred = model2.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [21]:
accuracy_mean = []
f1_mean = []
precision_mean = []
recall_mean = []

for i in range(0,len(columns_names.shape),1):
    accuracy_mean.append(accuracy_score(y_test.iloc[:,i], y_pred.iloc[:,i]))
    f1_mean.append(f1_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))
    precision_mean.append(precision_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))
    recall_mean.append(recall_score(y_test.iloc[:,i], y_pred.iloc[:,i],average='macro'))

accuracy_mean = pd.DataFrame(accuracy_mean)
f1_mean = pd.DataFrame(f1_mean)
precision_mean = pd.DataFrame(precision_mean)
recall_mean = pd.DataFrame(recall_mean)

print('The mean accuracy score of the model is',accuracy_mean.mean())
print('The mean precision score of the model is',precision_mean.mean())
print('The mean recall of the model is',recall_mean.mean())
print('The mean F1 score of the model is',f1_mean.mean())

The mean accuracy score of the model is 0    0.763921
dtype: float64
The mean precision score of the model is 0    0.639114
dtype: float64
The mean recall of the model is 0    0.401592
dtype: float64
The mean F1 score of the model is 0    0.408123
dtype: float64


### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model,open(model_filepath,'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.